In [ ]:
'''
‧Tutorial_8
‧記錄到資料庫
'''
import json
import configparser

from flask import Flask
from flask import request
from flask import make_response

import telegram

from util import Switch
from services import Services
from stock import Stock
from chatdb import ChatDB
from news import News
from help import Help

app = Flask(__name__)
#run_with_ngrok(app)   
#starts ngrok when the app is run
@app.route("/", methods=['GET'])
def hello():
    print(bot.get_me())
    return '您好，我是 vegetabot，很高興為您服務'

@app.route('/webhook', methods=['POST'])
def webhook():
    req = request.get_json(silent=True, force=True)
    print("Request:")
    print(json.dumps(req, indent=4))

    res = makeWebhookResult(req)

    res = json.dumps(res, indent=4)
    print(res)
    r = make_response(res)
    r.headers['Content-Type'] = 'application/json'
    return r

def askServices(req):
    services = Services(req)
    serviceType = services.type

    response = None
    for case in Switch(serviceType):
        if case('/help'):
            response = askHelp(req)
            break
        if case('/service'):
            print('詢問服務項目')
            response = services.inlineList(req, bot)
            break
        if case('news'):
            print('詢問新聞')
            response = askNews(req)
            break
        if case():
            print('無對應動作')

    return response
def askStock(req, bot):
    s = Stock(req)
    speech = s.idvStock(req)
    s.inlineList(req, bot)
    print("Response:")
    print(speech)
    #回傳
    return { 
      "textToSpeech": speech,
      "ssml": speech,
      "fulfillmentText": speech,
      "displayText": speech
    }

def askStockList(req):
    s = Stock(req)
    speech = s.stockList(req)
    print("Response:")
    print(speech)
    #回傳
    return { 
      "textToSpeech": speech,
      "ssml": speech,
      "fulfillmentText": speech,
      "displayText": speech
    }

def askStockChart(req, bot):
    
    context = req.get("queryResult").get("outputContexts")[0].get("parameters")
    response = None
    for case in Switch(context.get("Stock_Statistic")):
        if case('day statistic'):
            s = Stock(req)
            response = s.day(req,bot)
            break
        if case('month statistic'):
            s = Stock(req)
            response = s.month(req,bot)
            break
        if case('year statistic'):
            s = Stock(req)
            response = s.year(req,bot)
            break
        if case('Corporat'):
            s = Stock(req)
            response = s.Corporat(req,bot)
            break 
        if case():
            print('無對應動作')

    return response
def askNews(req):
    n = News(req)
    speech = n.yahoo()
    print("Response:")
    print(speech)
    #回傳
    return { 
      "textToSpeech": speech,
      "ssml": speech,
      "fulfillmentText": speech,
      "displayText": speech
    }
def askHelp(req):
    h = Help(req)
    speech = h.help()
    print("Response:")
    print(speech)
    #回傳
    return { 
      "textToSpeech": speech,
      "ssml": speech,
      "fulfillmentText": speech,
      "displayText": speech
    }
def doAction(req):
    response = None
    for case in Switch(req.get("queryResult").get("action")):
        
        if case('askServices'):
            print('詢問服務項目')
            response = askServices(req)
            break
        if case('askNews'):
            print('詢問新聞')
            response = askNews(req)
            break
        if case('askStocks'):
            print('詢問股票行情')
            response = askStock(req, bot)
            break
        if case('askStockChart'):
            print('查詢股票圖表')
            response = askStockChart(req, bot)
            break
        if case('help'):
            print()
        if case():
            print('無對應動作')

    return response

def reqRefactor(req):
    if req.get("originalDetectIntentRequest").get("payload").get("data").get("callback_query"):
        req["user"] = req.get("originalDetectIntentRequest").get("payload").get("data").get("callback_query")
    else:
        req["user"] = req.get("originalDetectIntentRequest").get("payload").get("data")

    if req.get("user").get("message"):
        req["user"]["chat"] = req.get("user").get("message").get("chat")

    return req

def recordToDB(req, res):
    db = ChatDB(config['mysql'])
    db.connect()
    
    last_name =  req.get("user").get("from").get("last_name")
    
    if last_name == None:
        last_name = ""
        
    user = {
        'user_id': req.get("user").get("from").get("id"),
        'chat_id': req.get("user").get("chat").get("id"), 
        'nicename': req.get("user").get("from").get("username"),
        'display_name': req.get("user").get("from").get("first_name") + " " + last_name
    }
    
    userID = db.find_user(user)
    if userID == None :
        userID = db.create_user(user)

    print("userID:" + str(userID))

    chat = {
        'chat_user': userID,
        'chat_ask': req.get("queryResult").get("queryText"),
        'bot_response': json.dumps(res, ensure_ascii=False),
        'bot_response_type': 'text', 
        'chat_parent': 0,
        'chat_intent': req.get("queryResult").get("intent").get("displayName"),
        'chat_action': req.get("queryResult").get("action"), 
        'chat_entity': json.dumps(req.get("queryResult").get("parameters"), ensure_ascii=False),
        'chat_context': json.dumps(req.get("queryResult").get("outputContexts"), ensure_ascii=False)
    }

    db.create_chat(chat)

    db.close()

def makeWebhookResult(req):
    req = reqRefactor(req)
    res = doAction(req)
    recordToDB(req, res)
    if res == None:
        return {}
    
    print("Response:")
    print(res)
    #回傳
    return res

if __name__ == "__main__":
    config = configparser.ConfigParser()
    config.read("config.ini", encoding="utf-8-sig")

    telegramToken = config['telegram']['Token']
    bot = telegram.Bot(token=telegramToken)

    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
